# Day 13: Shuttle Search

Your ferry can make it safely to a nearby port, but it won't get much further. When you call to book another ship, you discover that no ships embark from that port to your vacation island. You'll need to get from the port to the nearest airport.

Fortunately, a shuttle bus service is available to bring you from the sea port to the airport! Each bus has an ID number that also indicates how often the bus leaves for the airport.

Bus schedules are defined based on a timestamp that measures the number of minutes since some fixed reference point in the past. At timestamp 0, every bus simultaneously departed from the sea port. After that, each bus travels to the airport, then various other locations, and finally returns to the sea port to repeat its journey forever.

The time this loop takes a particular bus is also its ID number: the bus with ID 5 departs from the sea port at timestamps 0, 5, 10, 15, and so on. The bus with ID 11 departs at 0, 11, 22, 33, and so on. If you are there when the bus departs, you can ride that bus to the airport!

Your notes (your puzzle input) consist of two lines. The first line is your estimate of the earliest timestamp you could depart on a bus. The second line lists the bus IDs that are in service according to the shuttle company; entries that show x must be out of service, so you decide to ignore them.

To save time once you arrive, your goal is to figure out the earliest bus you can take to the airport. (There will be exactly one such bus.)

For example, suppose you have the following notes:

```text
939
7,13,x,x,59,x,31,19
```

Here, the earliest timestamp you could depart is 939, and the bus IDs in service are 7, 13, 59, 31, and 19. Near timestamp 939, these bus IDs depart at the times marked D:

```text
time   bus 7   bus 13  bus 59  bus 31  bus 19
929      .       .       .       .       .
930      .       .       .       D       .
931      D       .       .       .       D
932      .       .       .       .       .
933      .       .       .       .       .
934      .       .       .       .       .
935      .       .       .       .       .
936      .       D       .       .       .
937      .       .       .       .       .
938      D       .       .       .       .
939      .       .       .       .       .
940      .       .       .       .       .
941      .       .       .       .       .
942      .       .       .       .       .
943      .       .       .       .       .
944      .       .       D       .       .
945      D       .       .       .       .
946      .       .       .       .       .
947      .       .       .       .       .
948      .       .       .       .       .
949      .       D       .       .       .
```

The earliest bus you could take is bus ID 59. It doesn't depart until timestamp 944, so you would need to wait 944 - 939 = 5 minutes before it departs. Multiplying the bus ID by the number of minutes you'd need to wait gives 295.

What is the ID of the earliest bus you can take to the airport multiplied by the number of minutes you'll need to wait for that bus?

In [1]:
# Python imports
from math import prod
from pathlib import Path
from typing import List, Tuple

import numpy as np

We begin as usual with a function that loads input data into a useful form. Here, we load our timestamp (waiting start time), and the numbers of the buses that are available.

In [2]:
def load_data(fpath: str) -> Tuple[int, List[int]]:
    """Return the puzzle timestamp and bus number list
    
    :param fpath:  path to file containing puzzle input
    """
    with Path(fpath).open("r") as ifh:
        timestamp = int(ifh.readline().strip())
        buses = [int(_) for _ in ifh.readline().strip().split(",") if _ != "x"]
    
    return timestamp, buses

In [3]:
timestamp, buses = load_data("day13_test.txt")
timestamp, buses

(939, [7, 13, 59, 31, 19])

This is a number theory/prime number/modulus problem.

As buses leave on timestamps of multiples of their bus number, their leaving times are 0 (mod bus number). Any other time is the current timestamp (mod bus number) into the journey.

The journey takes (bus number) units of time, so the wait time is (bus number) - (current timestamp [mod bus number]).

We make the function `earliest_bus` calculate this value for all input buses, and return the shortest wait for any bus, and that bus' number.

In [4]:
def earliest_bus(timestamp: int, buses: List[int]) -> Tuple[int, int]:
    """Return shortest wait time for a bus, and that bus' number.
    
    :param timestamp:  current time (waiting start time)
    :param buses:  list of bus numbers
    """
    bus_waits = sorted([(bus - timestamp % bus, bus) for bus in buses])
    return bus_waits[0]

We solve the test problem.

In [5]:
wait, bus = earliest_bus(timestamp, buses)
wait, bus, wait * bus

(5, 59, 295)

Then the real problem.

In [6]:
timestamp, buses = load_data("day13_data.txt")
wait, bus = earliest_bus(timestamp, buses)
wait, bus, wait * bus

(5, 509, 2545)

## Part Two

The shuttle company is running a contest: one gold coin for anyone that can find the earliest timestamp such that the first bus ID departs at that time and each subsequent listed bus ID departs at that subsequent minute. (The first line in your input is no longer relevant.)

For example, suppose you have the same list of bus IDs as above:

```text
7,13,x,x,59,x,31,19
````

An `x` in the schedule means there are no constraints on what bus IDs must depart at that time.

This means you are looking for the earliest timestamp (called t) such that:

    Bus ID 7 departs at timestamp t.
    Bus ID 13 departs one minute after timestamp t.
    There are no requirements or restrictions on departures at two or three minutes after timestamp t.
    Bus ID 59 departs four minutes after timestamp t.
    There are no requirements or restrictions on departures at five minutes after timestamp t.
    Bus ID 31 departs six minutes after timestamp t.
    Bus ID 19 departs seven minutes after timestamp t.

The only bus departures that matter are the listed bus IDs at their specific offsets from t. Those bus IDs can depart at other times, and other bus IDs can depart at those times. For example, in the list above, because bus ID 19 must depart seven minutes after the timestamp at which bus ID 7 departs, bus ID 7 will always also be departing with bus ID 19 at seven minutes after timestamp t.

In this example, the earliest timestamp at which this occurs is 1068781:

```text
time     bus 7   bus 13  bus 59  bus 31  bus 19
1068773    .       .       .       .       .
1068774    D       .       .       .       .
1068775    .       .       .       .       .
1068776    .       .       .       .       .
1068777    .       .       .       .       .
1068778    .       .       .       .       .
1068779    .       .       .       .       .
1068780    .       .       .       .       .
1068781    D       .       .       .       .
1068782    .       D       .       .       .
1068783    .       .       .       .       .
1068784    .       .       .       .       .
1068785    .       .       D       .       .
1068786    .       .       .       .       .
1068787    .       .       .       D       .
1068788    D       .       .       .       D
1068789    .       .       .       .       .
1068790    .       .       .       .       .
1068791    .       .       .       .       .
1068792    .       .       .       .       .
1068793    .       .       .       .       .
1068794    .       .       .       .       .
1068795    D       D       .       .       .
1068796    .       .       .       .       .
1068797    .       .       .       .       .
```

In the above example, bus ID 7 departs at timestamp 1068788 (seven minutes after t). This is fine; the only requirement on that minute is that bus ID 19 departs then, and it does.

Here are some other examples:

    The earliest timestamp that matches the list 17,x,13,19 is 3417.
    67,7,59,61 first occurs at timestamp 754018.
    67,x,7,59,61 first occurs at timestamp 779210.
    67,7,x,59,61 first occurs at timestamp 1261476.
    1789,37,47,1889 first occurs at timestamp 1202161486.

However, with so many bus IDs in your list, surely the actual earliest timestamp will be larger than 100000000000000!

What is the earliest timestamp such that all of the listed bus IDs depart at offsets matching their positions in the list?

This is an extension of the above problem. The bus numbers are all prime, so the pattern of leave times repeats at the timestamp that is the product of all their numbers (and all multiples of this); this sets an upper limit on the appropriate timestamp.

Consider two bus numbers p and q, with p > q. Suppose p must have a wait time of s and q must have a wait time of t. If 0 is not a solution time, then all possible solution times are s (mod p). Such solution times must also be t (mod q); that is, s = t (mod pq).

This generalises so that all the wait times must be equal modulo the product of all bus numbers.

A nuance of this is that a bus' wait time in the input data may be greater than that bus' number. In these cases, we need to calculate the wait time as index (mod bus number).

> It took me quite a while to realise that the bus number could be greater than that bus' index in the list; that was my main solution bottleneck. When the index is greater than the bus number, my algorithm does not find a solution or halt.

We write a function `load_offsets()` that loads puzzle input data from a file and calculates the appropriate wait time.

In [7]:
def load_offsets(fpath: str) -> List[Tuple[int, int]]:
    """Return a list of (bus number, wait time) tuples from the puzzle input file
    
    :param fpath:  path to puzzle input file
    """
    with Path(fpath).open("r") as ifh:
        timestamp = int(ifh.readline().strip())
        buses = [(int(bus), idx % int(bus)) for idx, bus in enumerate(ifh.readline().strip().split(",")) if bus != "x"]
    
    return buses

In [8]:
buses = load_offsets("day13_test.txt")
buses

[(7, 0), (13, 1), (59, 4), (31, 6), (19, 7)]

We write the `check_buses()` function to check our candidate solutions.

In [9]:
def check_buses(timestamp: int, buses: List[Tuple[int, int]]) -> bool:
    """Returns True if timestamp results in the correct wait times
    
    :param timestamp:  candidate puzzle solution
    :param buses:  (bus number, wait time) puzzle inputs
    """
    bus_waits = [(bus, bus - timestamp % bus) for bus, wait in buses]
    bus_waits = [(bus, 0 if wait == bus else wait) for bus, wait in bus_waits]
    if bus_waits == buses:
        return True

The contest-winning algorithm uses the observation above to generate and test candidate timestamp solutions.

To minimise the search time, we sort buses in order of bus number, in order to exclude the largest number of candidate values as early as possible. This works because we test for the first solution mod p (where p is the biggest possible single modulus), then the second with mod pq (the biggest possible modulus that is the product of two bus numbers p and q), and so on until we are looking at a modulus that is the product of all bus numbers except the smallest. (This also implies a lower bound on the initial search value of that modulus but, empirically, the function is faster starting from zero).

We implement the algorithm in the function `find_wait_timestamp()`

In [10]:
def find_wait_timestamp(buses: List[Tuple[int, int]]) -> int:
    """Returns timestamp at which all buses have stated wait times
    
    :param buses:  list of (bus number, wait time) puzzle inputs
    """
    buses = sorted(buses, reverse=True)
    bcopy = buses[:]  # copy necessary for check as we pop buses from input list

    # Result must be in following limits
    lower_bound = prod([bus for bus, idx in buses[:-1]])
    upper_bound = prod([bus for bus, idx in buses])
    print(f"{lower_bound=} {upper_bound=}")
    
    # Start at time zero (not a solution), with timestamp step size 1
    curtime, curstep = 0, 1  # curtime is the candidate time; curstep the test modulus
    while len(buses):  # If there are any buses left in the list, get the next input
        curbus, curoffset = buses.pop(0)  # bus number and wait time
        
        # If the expected wait time is zero, set the expected wait time
        # to the bus number (=0 (mod bus number)); this is necessary for
        # the calculations/checks
        if curoffset == 0:
            curoffset = curbus
        print(f"{curbus=}, {curoffset=}, {curtime=}, {curstep=}")
        
        # Loop while the candidate time does not result in the expected
        # wait time (i.e. timestamp (mod curstep) is not wait time)
        while curbus - curtime % curbus != curoffset:
            curtime += curstep  # increment the candidate time equal to modulus
            if check_buses(curtime, bcopy):  # candidate is a winner?
                return curtime  # return the winning timestamp
        
        # The buses considered so far have the correct wait times, so
        # increase the modulus to the product of the current bus number
        # and the current modulus.
        # For instance, if current modulus is pq and current bus number is r,
        # new modulus is pqr
        # Our search excludes exponentially more search space with each new
        # bus added
        curstep *= curbus        

In [11]:
buses = load_offsets("day13_test.txt")
find_wait_timestamp(buses)

lower_bound=451763 upper_bound=3162341
curbus=59, curoffset=4, curtime=0, curstep=1
curbus=31, curoffset=6, curtime=55, curstep=59
curbus=19, curoffset=7, curtime=645, curstep=1829
curbus=13, curoffset=1, curtime=26251, curstep=34751
curbus=7, curoffset=7, curtime=165255, curstep=451763


1068781

In [12]:
find_wait_timestamp([(1789,0), (37,1), (47,2), (1889,3)])

lower_bound=158832787 upper_bound=5876813119
curbus=1889, curoffset=3, curtime=0, curstep=1
curbus=1789, curoffset=1789, curtime=1886, curstep=1889
curbus=47, curoffset=2, curtime=2467031, curstep=3379421
curbus=37, curoffset=1, curtime=90331977, curstep=158832787


1202161486

In [13]:
buses = load_offsets("day13_data.txt")
find_wait_timestamp(buses)

lower_bound=106965245506139 upper_bound=1390548191579807
curbus=643, curoffset=19, curtime=0, curstep=1
curbus=509, curoffset=50, curtime=624, curstep=643
curbus=41, curoffset=9, curtime=275828, curstep=327287
curbus=37, curoffset=19, curtime=2566837, curstep=13418767
curbus=29, curoffset=21, curtime=56241905, curstep=496494379
curbus=23, curoffset=19, curtime=8000151969, curstep=14398336991
curbus=19, curoffset=19, curtime=281568554798, curstep=331161750793
curbus=17, curoffset=2, curtime=1937377308763, curstep=6292073265067
curbus=13, curoffset=11, curtime=52273963429299, curstep=106965245506139


266204454441577